In [ ]:
## pipeline steps that have to happen for each new page:

# 1. determine which pages to exclude and why
# 2. write a <site>.py file implementing these exclusion rules
# 3. determine ideal update frequency? Or just go with 30 min?

# AJC

In [9]:
from playwright.async_api import async_playwright
import os

In [1]:
exclusion_string = 'https://www.ajc.com/gdpr.html'

In [5]:
file_to_load = '../scripts/tmp/web.archive.org/web/20230104004902/ajc.com/index.html'

In [7]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

In [8]:
await page.route(
    "**/*", 
    lambda route: route.abort() 
        if route.request.resource_type == "image" 
        else route.continue_() 
) 

await page.route(
    "https://web.archive.org*/*", 
    lambda route: route.abort() 
) 

In [21]:
from bs4 import BeautifulSoup
import re
soup = BeautifulSoup(open(file_to_load).read())

raw_text = soup.get_text()
text = re.sub('\s+', ' ', raw_text)

In [25]:
raw_text = soup.get_text()
text = re.sub('\s+', ' ', raw_text)

In [27]:
key_string = 'Redirecting to... https://www.ajc.com/gdpr.html' 
key_string in text

True

In [30]:
import pandas as pd 
import requests

In [32]:
resp = requests.get('http://web.archive.org/cdx/search/cdx?url=ajc.com&output=json&from=20230101&to=20230102')

In [41]:
resp_json = json.loads(resp.text)

In [43]:
import json
resp_df = pd.DataFrame(resp_json[1:], columns=resp_json[0])

In [45]:
resp_df.loc[lambda df: df['statuscode'] == '200']

,urlkey,timestamp,original,mimetype,statuscode,digest,length
4,"com,ajc)/",20230101005050,https://www.ajc.com/,text/html,200,PUF6V2YX5QDRUEOZJ4XYDVEJH5ACUR6C,367598
48,"com,ajc)/",20230101112349,https://www.ajc.com/,text/html,200,AWXOTPZCIV5223N43VWWIEH6Z3WVCQD2,318064
85,"com,ajc)/",20230101204845,https://www.ajc.com/,text/html,200,6GAG3G2C2HOEIERXSLIM7LXM4LROOJQO,397417
102,"com,ajc)/",20230102005131,https://www.ajc.com/,text/html,200,AYRY22KXD3A5IUOF4JPW6R2MZCQ5MAK7,409731
152,"com,ajc)/",20230102124629,https://www.ajc.com/,text/html,200,BRJHEQFEWC4QHL5WDOS636BANWQSQY6J,407993


In [47]:
resp_df.loc[lambda df: df['timestamp'] == '20230102004843']

,urlkey,timestamp,original,mimetype,statuscode,digest,length
101,"com,ajc)/",20230102004843,http://www.ajc.com/,unk,301,3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ,361


In [48]:
resp_df.loc[lambda df: df['timestamp'] == '20230102123352']

,urlkey,timestamp,original,mimetype,statuscode,digest,length
151,"com,ajc)/",20230102123352,http://www.ajc.com/,unk,301,3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ,372


In [ ]:
2023_01_01_00_50_50
2023_01_01_11_23_49
2023_01_01_20_48_45
2023_01_02_00_48_43
2023_01_02_12_33_52

In [49]:
top_10_georgia_news = [
    'http://www.ajc.com/',                # skips many based on 
    'http://www.gwinnettdailypost.com',   # all good - dynamic resizing
    'http://savannahnow.com',             # all good - dynamic resizing
    'http://www.ledger-enquirer.com',     # all good - dynamic resizing
    'https://www.augustachronicle.com/',  # looks good 
    'https://www.macon.com/',             # will have to do some clustering — looks like some are missing CSS files.
    'http://www.albanyherald.com',        # 
    'http://www.gainesvilletimes.com',
    'http://www.thebrunswicknews.com',
    'http://onlineathens.com',
]

# Slosh

In [52]:
sites_df = pd.read_csv('../bin/news-homepages/newshomepages/sources/sites.csv')

In [59]:
sites_df['bundle'].str.split('|', expand=True).unstack().dropna().value_counts().head()

gannett       239
california     54
texas          43
georgia        38
illinois       36
dtype: int64

In [60]:
sites_df

,handle,url,name,location,timezone,country,language,bundle,wait
0,Hou_Landing,https://www.houstonlanding.org/,Houston Landing,Houston,America/Chicago,US,en,texas,NaN
1,WBTV_News,https://www.wbtv.com/,WBTV,Charlotte,America/New_York,US,en,north-carolina,NaN
2,signalcleveland,https://signalcleveland.org/,Signal Cleveland,Cleveland,America/New_York,US,en,NaN,NaN
3,100Reporters,http://100r.org/,100Reporters,Washington,America/New_York,US,en,NaN,NaN
4,11AliveNews,https://www.11alive.com,11Alive News,Atlanta,America/New_York,US,en,georgia,NaN
...,...,...,...,...,...,...,...,...,...
1148,theolympian,http://www.theolympian.com,The Olympian,Olympia,America/Los_Angeles,US,en,washington|mcclatchy,NaN
1149,thestate,http://www.thestate.com,The State,"1927 Thurmond Mall, Suite 102, Columbia, SC 29201",America/New_York,US,en,south-carolina|mcclatchy,NaN
1150,TheSunNews,http://www.myrtlebeachonline.com,Sun News,Myrtle Beach,America/New_York,US,en,south-carolina|mcclatchy,NaN
1151,TriCityHerald,http://www.tri-cityherald.com,Tri-City Herald,Kennewick,America/Los_Angeles,US,en,washington|mcclatchy,NaN


In [64]:
sites_df.loc[lambda df: df['bundle'].fillna('').apply(lambda x: 'georgia' in x)]

,handle,url,name,location,timezone,country,language,bundle,wait
4,11AliveNews,https://www.11alive.com,11Alive News,Atlanta,America/New_York,US,en,georgia,NaN
7,13wmaznews,https://www.13wmaz.com,13WMAZ News,Macon,America/New_York,US,en,georgia,NaN
13,41NBC,https://www.41nbc.com,41NBC / WMGT,Macon,America/New_York,US,en,georgia,NaN
27,AUG_Chronicle,https://www.augustachronicle.com,Augusta Chronicle,Augusta,America/New_York,US,en,georgia,NaN
34,Albany_Herald,https://www.albanyherald.com,Albany Herald,Albany,America/New_York,US,en,georgia,NaN
87,Brunswick_News,https://www.thebrunswicknews.com,The Brunswick News,Brunswick,America/New_York,US,en,georgia,NaN
112,CapitalB_ATL,https://atlanta.capitalbnews.org/,Capital B Atlanta,Atlanta,America/New_York,US,en,georgia,NaN
203,Fox5Atlanta,https://www.fox5atlanta.com,Fox 5 Atlanta,Atlanta,America/New_York,US,en,georgia,NaN
218,GwinnettDaily,https://www.gwinnettdailypost.com,Gwinnett Daily Post,Lawrenceville,America/New_York,US,en,georgia,NaN
304,LagrangeNews,https://www.lagrangenews.com,The LaGrange Daily News,LaGrange,America/New_York,US,en,georgia,NaN
